<div>
    <h1>Large Language Models Projects</a></h1>
    <h3>Apply and Implement Strategies for Large Language Models</h3>
    <p>by <b>Pere Martra</b></p>
    <h2>3.4-Create a Medical Assistant RAG System chat with LangChain & ChromaDB</h2>
</div>



#Installing libraries & Loading Dataset

We will download the dataset from the Hugging Face datasets library. It's a dataset with information about diseases.

In [1]:
from datasets import load_dataset

data = load_dataset("keivalya/MedQuad-MedicalQnADataset", split='train')


In [2]:
data = data.to_pandas()
data.head(10)

,qtype,Question,Answer
0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."
5,prevention,How to prevent Lymphocytic Choriomeningitis (L...,LCMV infection can be prevented by avoiding co...
6,information,What is (are) Parasites - Cysticercosis ?,Cysticercosis is an infection caused by the la...
7,susceptibility,Who is at risk for Parasites - Cysticercosis? ?,Cysticercosis is an infection caused by the la...
8,exams and tests,How to diagnose Parasites - Cysticercosis ?,"If you think that you may have cysticercosis, ..."
9,treatment,What are the treatments for Parasites - Cystic...,Some people with cysticercosis do not need to ...


In [3]:
#uncoment this line if you want to limit the size of the data.
data = data[0:100]

As you can see, the medical information in the dataset is well-organized, and to someone like me, who is not an expert in the field, it appears to be quite valuable. This information could be a useful addition to any general medicine book to support primary care doctors.

Load the langchain libraries to load the document.

In [4]:
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma

The Document is in the Answer column, and the others columns are Metadata.

In [5]:
df_loader = DataFrameLoader(data, page_content_column="Answer")


In [6]:
df_document = df_loader.load()
display(df_document[:2])

[Document(page_content='LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents.  Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent. Person-to-person transmission has not been reported, with the exception of vertical transmission from infected mother to fetus, and rarely, through organ transplantation.', metadata={'qtype': 'susceptibility', 'Question': 'Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?'}),
 Document(page_content='LCMV is most commonly recognized as causing neurological disease, as its name implies, though infection without symptoms or mild febrile illnesses are more common clinical manifestations. \n                \nFor infected persons who do become ill, onset of symptoms usually occurs 8-13 days after exposure to the virus as part of a biphasic febrile illness. This initial 

We can chunk the documents. The size to which we want to split the document is a design decision. The larger it is, the larger the prompt will be, and the slower the Model's response process.

We also need to consider the maximum prompt size and ensure that the document does not exceed it.

In [7]:
from langchain.text_splitter import CharacterTextSplitter

In [8]:
text_splitter = CharacterTextSplitter(chunk_size=1250,
                                      separator="\n",
                                      chunk_overlap=100)
texts = text_splitter.split_documents(df_document)


These warnings we see are because it can't perform the partition of the required size. This is because it waits for a page break to divide the text and does so when possible.

In [9]:
first_doc = texts[1]
print(first_doc.page_content)

LCMV is most commonly recognized as causing neurological disease, as its name implies, though infection without symptoms or mild febrile illnesses are more common clinical manifestations. 
                
For infected persons who do become ill, onset of symptoms usually occurs 8-13 days after exposure to the virus as part of a biphasic febrile illness. This initial phase, which may last as long as a week, typically begins with any or all of the following symptoms: fever, malaise, lack of appetite, muscle aches, headache, nausea, and vomiting. Other symptoms appearing less frequently include sore throat, cough, joint pain, chest pain, testicular pain, and parotid (salivary gland) pain.


### Initialize the Embedding Model and Vector DB

We load the text-embedding-ada-002 model from OpenAI.

In [12]:
from utils import model_chooser
chooser= model_chooser()

In [14]:
from langchain_community.embeddings import OllamaEmbeddings
embed = OllamaEmbeddings(model=chooser.Model.value)

The execution of this cell may take 3 to 5 minutes. If you want it to be faster, you can reduce the number of records in the dataset.

In [15]:
directory_cdb = './chromadb'
chroma_db = Chroma.from_documents(
    df_document, embed, persist_directory=directory_cdb
)

We are going to create three objects.

* The language model, which can be any of those from OpenAI, the most common being gpt-3.5.
* The memory, responsible for keeping the prompt with all the necessary history.
* The retrieval, used to obtain information stored in ChromaDB.

In [16]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

from langchain_community.llms import Ollama

llm = Ollama(model=chooser.Model.value)

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=4, #Number of messages stored in memory
    return_messages=True #Must return the messages in the response.
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=chroma_db.as_retriever()
)

We can try the isolated Retrieval to see if the information it returns is relevant.




In [17]:
qa.run("What is the main symptom of LCM?")

/work/06659/wmobley/ls6/miniconda3/envs/llm/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


" I'm sorry for any confusion, but the context provided does not mention specific symptoms associated with LCMV (Lymphocytic Choriomeningitis Virus) infections. The text only describes the ways in which LCMV infections can be transmitted. If you have more context or information about LCMV symptoms, I'd be happy to help further interpret that.\n\nAs for Acanthamoeba, it can cause severe illnesses such as Acanthamoeba keratitis (an eye infection), Granulomatous Encephalitis (an infection of the brain and spinal cord), and disseminated infections that spread throughout the body."

Perfect! The information returned is exactly what we desired.

## Creating the Agent.

In [18]:
from langchain.agents import Tool

#Defining the list of tool objects to be used by LangChain.
tools = [
    Tool(
        name='Medical KB',
        func=qa.run,
        description=(
            """use this tool when answering medical knowledge queries to get
            more information about the topic"""
        )
    )
]

In [19]:
from langchain.agents import create_react_agent
from langchain import hub

prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(
    tools=tools,
    llm=llm,
    prompt=prompt,
)

In [20]:
# Create an agent executor by passing in the agent and tools
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               memory=conversational_memory,
                               max_iterations=30,
                               max_execution_time=600,
                               handle_parsing_errors=True
                               )

### Using the Conversational Agent

To make queries we simply call the `agent` directly.

First i will try a order not related to the Medical field.

In [21]:
agent_executor.invoke({"input": "Give me the area of square of 2x2"})



> Entering new AgentExecutor chain...
 Thought: Do I need to use a tool? No
Final Answer: The area of a square with side length 2x2 is (2x2)^2 = 4x2^2 = 8x2 square units.

> Finished chain.


{'input': 'Give me the area of square of 2x2',
 'chat_history': [],
 'output': 'The area of a square with side length 2x2 is (2x2)^2 = 4x2^2 = 8x2 square units.'}

Perfect, the model has responded without accessing the configured knowledge database.

Now I will try with a question that is also not related to health.

In [22]:
agent_executor.invoke({"input": "Do you know who is Clark Kent?"})



> Entering new AgentExecutor chain...
 Thought: Do I need to use a tool? No
Final Answer: Yes, I can provide information on that. Clark Kent is a fictional character in the DC Comics universe. He is the secret identity of Superman, a superhero who was born on the planet Krypton and was raised on Earth. In his Superman persona, he is known for having a wide range of superpowers, including super strength, speed, invulnerability, flight, and heat vision. As Clark Kent, he works as a journalist for the Daily Planet newspaper in Metropolis, where he often finds himself in situations that require the intervention of Superman.

> Finished chain.


{'input': 'Do you know who is Clark Kent?',
 'chat_history': [HumanMessage(content='Give me the area of square of 2x2'),
  AIMessage(content='The area of a square with side length 2x2 is (2x2)^2 = 4x2^2 = 8x2 square units.')],
 'output': 'Yes, I can provide information on that. Clark Kent is a fictional character in the DC Comics universe. He is the secret identity of Superman, a superhero who was born on the planet Krypton and was raised on Earth. In his Superman persona, he is known for having a wide range of superpowers, including super strength, speed, invulnerability, flight, and heat vision. As Clark Kent, he works as a journalist for the Daily Planet newspaper in Metropolis, where he often finds himself in situations that require the intervention of Superman.'}

It has not accessed either, as the model has been able to identify that it is not a question related to the database that LangChain provides.

Now it's time to try with a question related to Medicine. Let's see if the model can understand that it should first look for information in the vector database at its disposal.

In [23]:
 agent_executor.memory.clear()

In [24]:
agent_executor.invoke({"input": """I have a patient that can have Botulism,
how can I confirm the diagnosis?"""})



> Entering new AgentExecutor chain...
 Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: symptoms and diagnostic tests for Botulism I don't have the information about the symptoms and diagnostic tests for botulism as it was not provided in the context. The text provided focuses on the diagnosis and transmission of Taenia tapeworm infections, including T. saginata, T. solium, and T. asiatica, as well as the disease cysticercosis which is caused by the ingestion of T. solium tapeworm eggs. Botulism is a different illness, caused by toxins produced by the bacterium Clostridium botulinum. It's important to refer to appropriate and credible sources for information about botulism symptoms and diagnostic tests. Do I need to use a tool? No

Final Answer: To confirm a diagnosis of botulism, a healthcare provider will typically consider the patient's symptoms and examine any food that the patient has recently eaten. In addition, they may perform certain diagnostic tests su

{'input': 'I have a patient that can have Botulism,\nhow can I confirm the diagnosis?',
 'chat_history': [],
 'output': "To confirm a diagnosis of botulism, a healthcare provider will typically consider the patient's symptoms and examine any food that the patient has recently eaten. In addition, they may perform certain diagnostic tests such as a nerve conduction study or a blood test to detect the presence of the botulinum toxin or evidence of bacterial growth. However, it is important to note that this is a general guideline and the specific steps for diagnosing botulism can vary depending on the individual case. If you suspect a patient has botulism, it is crucial to seek immediate medical attention as this is a serious condition that requires prompt treatment."}

Perfect, the most important thing for us is that it has been able to identify that it should go to the medical database to search for information about the symptoms.

In [25]:
agent_executor.invoke({"input": "Is this an important illness?"})



> Entering new AgentExecutor chain...
 Thought: Do I need to use a tool? No

Final Answer: Yes, botulism is considered an important illness. It is a rare but serious condition caused by exposure to the botulinum toxin, which can lead to paralysis and other severe symptoms if not promptly treated. If you suspect someone has botulism, it is crucial to seek medical attention immediately.

> Finished chain.


{'input': 'Is this an important illness?',
 'chat_history': [HumanMessage(content='I have a patient that can have Botulism,\nhow can I confirm the diagnosis?'),
  AIMessage(content="To confirm a diagnosis of botulism, a healthcare provider will typically consider the patient's symptoms and examine any food that the patient has recently eaten. In addition, they may perform certain diagnostic tests such as a nerve conduction study or a blood test to detect the presence of the botulinum toxin or evidence of bacterial growth. However, it is important to note that this is a general guideline and the specific steps for diagnosing botulism can vary depending on the individual case. If you suspect a patient has botulism, it is crucial to seek immediate medical attention as this is a serious condition that requires prompt treatment.")],
 'output': 'Yes, botulism is considered an important illness. It is a rare but serious condition caused by exposure to the botulinum toxin, which can lead to pa

And the memory works perfectly. We can maintain a conversation, taking into account that the model knows the previous questions and answers.

# Conclusions.
The experiment has been a small success. The Vectorial database has been configured and filled with information from the dataset. A LangChain agent has been created, and it has been able to retrieve information from the database only when necessary. Don't forget that our ChatBot has memory.

All of this in just a few lines of code!


---